# Import & Load Data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
df.head()

In [ ]:
df.info()

# Format 

the notes[description] of each patient are written in patient_Notes.csv while the main feature from that description is written in train.csv given the note we need an NER approach to find the location of the main features or main key phrase in that text the location is stored as `[starting_character_index_of_main_feat_in_the_text:ending_character_index_of_main_feat_in_the_text]` and we will see this in a moment so dont worry if u didnt get it.



In [ ]:
notes=pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
notes.head()

look here the location for this note was ['696 724'] u can see it in the train.csv head listed above so if crop that location i will get the feat

In [ ]:
notes['pn_history'][16][696:724]

# Our texts are bit messy lets clean them up

In [ ]:
notes['pn_history'][0]

In [ ]:
def remove(text,prefixes):
    for v in prefixes:text=text.replace(v,'')
    return text
cleaned=remove(notes['pn_history'][0],['\r','\n',':'])

In [ ]:
cleaned

looks like we can divide the sentence with the `,` prefix each having one important note like `physical examination, and treatment-began 2-3 months ago` so we can split this with `,`

In [ ]:
cleaned.split(',')

# seems great !

# Overview of patient_notes

In [ ]:
notes['feats']=notes['pn_history'].apply(lambda x:len(x.split(',')))
notes.head()

# Distribution of the number of parts we get when split by `,` operator

In [ ]:
px.histogram(notes['feats'])

In [ ]:
notes['words']=notes['pn_history'].apply(lambda x:len(x.split()))
notes.head()

# Distribution of number of words we get

In [ ]:
px.histogram(notes['words'])

# ratio between the two

In [ ]:
px.histogram(notes['words']/notes['feats'])

In [ ]:
df['no_of_feats']=df['annotation'].apply(lambda x:len(x.split(',')))
df.head()

# Histogram of number of features per note

In [ ]:
px.histogram(df['no_of_feats'])

# Length of each feature

In [ ]:
def str2vec(string):
    k=string[1:-1]
    k=k.replace("'",'')
    return k.split(',')
str2vec("['1','2','3']")

In [ ]:
features=list(map(str2vec,df['annotation']))
features[0]

In [ ]:

k=[]
for g in features:
    for l in g:
        k.append(len(l))
px.histogram(np.array(k))

# Most Common places for NER

In [ ]:
def span(location):
    listed=str2vec(location)

    listed=np.array(list(map(lambda x:x.split(),listed)))
    try:

        return list(map(int,list(listed.T[0]))),list(map(int,list(listed.T[1])))
    except:
        return list(),list()
span("['1 2','4 3']")

In [ ]:
train = df.merge(notes,on=['case_num','pn_num'])

In [ ]:
train.head()

In [ ]:
x1,y1=list(),list()
locations=list(map(span,train['location']))
lengths=np.array(list(map(len,train['pn_history'])))
i=0
for value in locations:
    x1+=list(np.array(value[0])/lengths[i])
   
    y1+=list(np.array(value[1])/lengths[i])
    i+=1

In [ ]:
x1,y1=np.array(x1),np.array(y1)

In [ ]:
fig = ff.create_2d_density(
    x1[:2000], y1[:2000], colorscale=colorscale,
    hist_color='rgb(255, 237, 222)', point_size=10
)

fig.show()

# Thank You Upvote if you Like Its to be noted that a lot of work is still in progress 
